English, Urdu, Hindi, Chinese, etc. supported language

In [2]:
pip install vosk soundfile

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 75.5 MB/s eta 0:00:00
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22427 sha256=6a161508b39b2f671f89f8d5ddf3ae96ab11de02de66fbb64102da6a298b3eae
  Stored in directory: /root/.cache/pip/wheels/1f/43/f1/23ee9119497fcb57d9f7046fbf34c6d9027c46a1fa7824cf08
Successfully built srt


In [4]:
!wget https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip

--2025-07-04 09:02:39--  https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41205931 (39M) [application/zip]
Saving to: ‘vosk-model-small-en-us-0.15.zip’

vosk-model-small-en 100%[===================>]  39.30M  13.2MB/s    in 3.0s    

2025-07-04 09:02:43 (13.2 MB/s) - ‘vosk-model-small-en-us-0.15.zip’ saved [41205931/41205931]



In [6]:
!unzip vosk-model-small-en-us-0.15.zip

Archive:  vosk-model-small-en-us-0.15.zip
   creating: vosk-model-small-en-us-0.15/
   creating: vosk-model-small-en-us-0.15/am/
  inflating: vosk-model-small-en-us-0.15/am/final.mdl  
   creating: vosk-model-small-en-us-0.15/graph/
  inflating: vosk-model-small-en-us-0.15/graph/disambig_tid.int  
  inflating: vosk-model-small-en-us-0.15/graph/HCLr.fst  
  inflating: vosk-model-small-en-us-0.15/graph/Gr.fst  
   creating: vosk-model-small-en-us-0.15/graph/phones/
  inflating: vosk-model-small-en-us-0.15/graph/phones/word_boundary.int  
   creating: vosk-model-small-en-us-0.15/conf/
  inflating: vosk-model-small-en-us-0.15/conf/model.conf  
  inflating: vosk-model-small-en-us-0.15/conf/mfcc.conf  
   creating: vosk-model-small-en-us-0.15/ivector/
  inflating: vosk-model-small-en-us-0.15/ivector/splice.conf  
  inflating: vosk-model-small-en-us-0.15/ivector/final.dubm  
  inflating: vosk-model-small-en-us-0.15/ivector/global_cmvn.stats  
  inflating: vosk-model-small-en-us-0.15/ivector/f

In [8]:
import time
import wave
import json
from vosk import Model, KaldiRecognizer
from pydub import AudioSegment

# === Convert MP3 to WAV ===
mp3_path = "/content/audio.mp3"
wav_path = "/content/audio.wav"

sound = AudioSegment.from_file(mp3_path)
sound = sound.set_frame_rate(16000).set_channels(1)
sound.export(wav_path, format="wav")

# === Vosk Model Path ===
MODEL_PATH = "vosk-model-small-en-us-0.15"
model = Model(MODEL_PATH)

# === Load WAV ===
wf = wave.open(wav_path, "rb")
assert wf.getnchannels() == 1
assert wf.getsampwidth() == 2
assert wf.getframerate() == 16000

# === Audio duration ===
frames = wf.getnframes()
rate = wf.getframerate()
duration = frames / float(rate)

# === Recognizer ===
rec = KaldiRecognizer(model, rate)

# === Transcribe + Measure Latency ===
start_time = time.time()
results = []

while True:
    data = wf.readframes(4000)
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        results.append(json.loads(rec.Result()))
results.append(json.loads(rec.FinalResult()))
end_time = time.time()

# === Output ===
transcription = " ".join([res.get("text", "") for res in results])
latency = end_time - start_time
rtf = latency / duration

print(f"📝 Transcription: {transcription}")
print(f"🎧 Duration: {duration:.2f} seconds")
print(f"⏱️ Latency: {latency:.2f} seconds")
print(f"⚡ RTF: {rtf:.2f}")

📝 Transcription: to check the latency score of your grope whisper large v three model transcription or any other speech to text model you can measure the in france duration and compare it to the length of the audio to compute the real time factor rt f
🎧 Duration: 15.24 seconds
⏱️ Latency: 2.63 seconds
⚡ RTF: 0.17
